In [3]:
from pyspark.sql import Row
from pyspark.sql.functions import col 
from pyspark.ml.linalg import Vectors

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.clustering import KMeans

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

import numpy as np
import time

In [ ]:
base_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_prediction/"

train_positive_pairs = sc.textFile(base_path + "new_positive_pairs_features/day*/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))
train_unlabled_pairs = sc.textFile(base_path + "new_unlabled_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))

test_positive_pairs = sc.textFile(base_path + "test_positive_pairs/part-*")

test_pairs_features = sc.textFile(base_path + "new_test_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))

test_pairs_features_50w = sc.textFile(base_path + "test_pairs_features_50w/part-*").map(lambda x: eval(x))

In [ ]:
print(train_unlabled_pairs.first())

In [ ]:
train_positive_pairs_np = np.array(train_positive_pairs.map(lambda x: Vectors.dense(x[1])).collect())

In [ ]:
train_positive_pairs_center = np.mean(train_positive_pairs_np, axis=0)

In [ ]:
print(train_positive_pairs_center)

In [ ]:
train_unlabled_pairs_df = spark.createDataFrame(train_unlabled_pairs.map(lambda x: Vectors.dense(x[1])).collect(), ["features"])

In [ ]:
k-means = KMeans(k=20, seed=1).fit(train_unlabled_pairs_df)

In [ ]:
centers = k-means.clusterCenters()

In [ ]:
print(centers)

In [21]:
data = [(Vectors.dense([0.0, 0.0]),), (Vectors.dense([1.0, 1.0]),), (Vectors.dense([4.0, 5.0]),), (Vectors.dense([9.0, 8.0]),), (Vectors.dense([8.0, 9.0]),)]
df = spark.createDataFrame(data, ["features"])
bkm = BisectingKMeans(k=2, minDivisibleClusterSize=1.0)
model = bkm.fit(df)
centers = model.clusterCenters()

In [27]:
centers

[array([ 1.66666667,  2.        ]), array([ 8.5,  8.5])]

In [17]:
kmeans = KMeans(k=2, seed=1).fit(df)

In [18]:
kmeans

[array([ 0.5,  0.5]), array([ 7.        ,  7.33333333])]

In [23]:
KMeans(k=3, seed=1).fit(df).clusterCenters()

[array([ 8.5,  8.5]), array([ 0.5,  0.5]), array([ 4.,  5.])]

In [ ]:
from scipy.spatial import distance

In [6]:
from pyspark.mllib.clustering import KMeans, KMeansModel

In [4]:
data = np.array([0.0,0.0, 1.0,1.0, 9.0,8.0, 8.0,9.0]).reshape(4, 2)

In [72]:
model = KMeans.train(sc.parallelize(data), 5)

In [65]:
k = 2
cluster_dict = {}
for x in range(k):
    cluster_dict[x] = 0
def f(x):
    cluster_dict[model.predict(x)] += 1
    return cluster_dict

In [77]:
a = sc.parallelize(data).map(lambda x: (model.predict(x), 1)).reduceByKey(lambda x, y : x + y).collect()

In [67]:
cluster_dict

{0: 0, 1: 0}

In [80]:
sorted(a, reverse=True)

[(3, 1), (2, 1), (1, 1), (0, 1)]